# Modelos para pronóstico de cuenta corriente (de series de tiempo y de ciencia de datos)

Debido a que los indicadores que componen la cuenta corriente se encuentran en constante revisión, las cuales pueden ser considerables, es necesario guardar las diferentes versiones con las que se corren los modelos para pronosticarla.

### Paso 1. Descargar la versión más reciente de la cuenta corriente de Banco de México

In [64]:
import sys
import numpy as np
import pandas as pd
from fun_descarga import *

In [65]:
c_corr=descargar_banxico('SE44352','2018-01-01','2023-06-01')
c_corr_credito=descargar_banxico('SE44353,SE44354,SE44357,SE44363,SE44369','2018-01-01','2023-06-01')
c_corr_debito=descargar_banxico('SE44372,SE44373,SE44376,SE44382,SE44390','2018-01-01','2023-06-01')
c_cap=descargar_banxico('SE44393,SE44394,SE44395','2018-01-01','2023-06-01')

In [69]:
#función que acorta los títulos específicos de balanza de pagos
def limpiar_titulo(c_corr_credito):
    for k,v in c_corr_credito.items():
        nueva=k.split("(a partir de 2002)")[1]
        c_corr_credito[nueva]=c_corr_credito.pop(k)
    return c_corr_credito
c_corr_credito=limpiar_titulo(c_corr_credito)
c_corr_debito=limpiar_titulo(c_corr_debito)


In [70]:
#ejemplo de cómo se ven los títulos más cortos
c_corr_credito.head()

,Cuenta corriente (I - II) I. Crédito (A + B + C + D) A. Bienes,Cuenta corriente (I - II) I. Crédito (A + B + C + D) D. Ingreso secundario,Cuenta corriente (I - II) I. Crédito (A + B + C + D) B. Servicios 2/,Cuenta corriente (I - II) I. Crédito (A + B + C + D) C. Ingreso primario,Cuenta corriente (I - II) I. Crédito (A + B + C + D)
fecha,,,,,
01/01/2018,105384.981986,7397.283475,10134.791731,3027.747818,125944.805010
01/04/2018,113971.903988,9271.883676,9462.266296,3584.743845,136290.797805
01/07/2018,114083.889374,8671.781192,9325.348179,3649.303030,135730.321775
01/10/2018,117641.459404,9185.096001,10690.699474,3270.704511,140787.959390
01/01/2019,108174.871901,8153.444736,11342.006534,4147.530939,131817.854110


### 2. Descargar variables explicativas: remesas, inversión fija bruta y servicio de deuda

### 3. Explicación de modelos autoarima, redes neuronales, crossval, inferencia bayesiana (quizá gridsearch)
Justificar selección de parámetros 

### 4. Modelos de predicción para cuenta corriente considerados

#### 4.1. Modelo autoarima

#### 4.2. Modelo de redes neuronales

#### 4.3. Modelo de inferencia bayesiana

### 5. Conclusiones